# Модель машинного обучения 

In [5]:
from python_files.data import data
from python_files.elasticity.ela_col_list import ela_x_col_list, ela_col_list_norm, ela_col_list_std
from python_files.elasticity.ela_split_data import X_train_ela, X_test_ela, y_train_ela, y_test_ela
# from python_files.elasticity.ela_outliers import X_train_trimmed_3S, y_train_trimmed_3S
# from python_files.elasticity.ela_outliers import X_train_trimmed_QT, y_train_trimmed_QT
# from python_files.elasticity.ela_outliers import X_train_sub_bound_3S, y_train_sub_bound_3S
# from python_files.elasticity.ela_outliers import X_train_sub_bound_QT, y_train_sub_bound_QT
# from python_files.elasticity.ela_outliers import X_train_sub_med_3S, y_train_sub_med_3S
# from python_files.elasticity.ela_outliers import import X_train_sub_med_QT, y_train_sub_med_QT

In [6]:
import numpy as np
import pandas as pd
import pickle

In [7]:
from sklearn.preprocessing import PowerTransformer, StandardScaler, QuantileTransformer, MaxAbsScaler, RobustScaler
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge
from sklearn.svm import SVR

## Preprocessing

In [8]:
preprocessor = ColumnTransformer(
    transformers=[
        ('normalization', PowerTransformer(), ela_col_list_norm), 
        ('scaling', StandardScaler(), ela_col_list_std) 
    ])

In [9]:
preprocessor_ = ColumnTransformer(
    transformers=[
        #('normalization', PowerTransformer(), ela_x_col_list)
        ('scaling', RobustScaler(), ela_x_col_list)
    ])

In [10]:
preprocessor_MAS = Pipeline(steps = [('scaling_1', StandardScaler(with_mean=True, with_std=False)),
                                     ('scaling_2', MaxAbsScaler())
                                    ])

## Model

In [5]:
# model = GradientBoostingRegressor(learning_rate = 0.01, n_estimators = 100, random_state = 7)

In [42]:
# model = ElasticNet(alpha = 0.2, l1_ratio = 0.4, random_state = 7)

In [61]:
# model = Lasso(alpha = 0.4, max_iter = 1000, random_state = 7) 

In [16]:
# model = RandomForestRegressor(random_state = 7, min_samples_split = 300, n_estimators = 300)

In [44]:
# model = SVR(kernel = 'linear', C = 5, gamma = 'scale')

In [ ]:
# model = Ridge(alpha = 2136, positive = False) 

In [11]:
model = TransformedTargetRegressor(regressor = Ridge(alpha = 2136, positive = False),
                                  transformer = QuantileTransformer(n_quantiles = 716, output_distribution="normal", random_state = 7))
                                   #func = np.log, inverse_func = np.exp)

In [12]:
Regressor = Pipeline(steps=[('preprocessor', preprocessor_),
                      ('model', model)],
                     verbose=True)

## Training

In [13]:
Regressor.fit(X_train_ela, y_train_ela)

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   6.2s
[Pipeline] ............. (step 2 of 2) Processing model, total=   4.6s


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('scaling', RobustScaler(),
                                                  Index(['Соотношение матрица-наполнитель', 'Плотность, кг/м3',
       'модуль упругости, ГПа', 'Количество отвердителя, м.%',
       'Содержание эпоксидных групп,%_2', 'Температура вспышки, С_2',
       'Поверхностная плотность, г/м2', 'Прочность при растяжении, МПа',
       'Потребление смолы, г/м2', 'Угол нашивки, град', 'Шаг нашивки',
       'Плотность нашивки'],
      dtype='object'))])),
                ('model',
                 TransformedTargetRegressor(regressor=Ridge(alpha=2136),
                                            transformer=QuantileTransformer(n_quantiles=716,
                                                                            output_distribution='normal',
                                                                            random_state=7)))],
         verbose=True)

## Predictions

In [14]:
predictions = Regressor.predict(X_test_ela)

## Evaluation

In [15]:
r2_score(y_test_ela, predictions)

0.0001408331849468425

In [16]:
np.sqrt(mean_squared_error(y_test_ela, predictions))

3.0283025335178526

In [17]:
mean_absolute_error(y_test_ela, predictions)

2.443540351805894

#### Пользовательские метрики

In [18]:
(1 - (mean_absolute_error(y_test_ela, predictions) / y_test_ela.mean())) * 100

96.66395125536484

In [19]:
mean_absolute_error(y_test_ela, predictions) * 100 / (y_test_ela.max() - y_test_ela.min())

14.265753933925408

In [20]:
y_test_ela.describe()

count    307.000000
mean      73.246542
std        3.033460
min       65.553336
25%       71.367894
50%       73.015274
75%       75.173719
max       82.682051
Name: Модуль упругости при растяжении, ГПа, dtype: float64

## Saving model

In [19]:
pickle.dump(Regressor, open('ela_regr_model.pkl', 'wb')) # wb - формат записи

In [21]:
loaded_ela_model = pickle.load(open('ela_regr_model.pkl', 'rb')) # rb - формат чтения

In [29]:
loaded_ela_model.predict(X_test_ela.iloc[100 : 100 + 1])

array([73.19419124])

In [30]:
Regressor.predict(X_test_ela.iloc[100 : 100 + 1])

array([73.19419124])